In [1]:
import selenium
import os
from selenium import webdriver
import time
import io
import requests
from PIL import Image
import hashlib

In [2]:
DRIVER_PATH ='C:/Users/Voence/Desktop/python_work/Scraping/chromedriver_win32/chromedriver.exe'

In [3]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)
    
    
    # build the google query
            
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [4]:
  def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [5]:
def search_and_download(search_term:str,driver_path:str,target_path='C:/Users/Voence/Desktop/python_work/Scraping/images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [7]:
search_term ='Elon Musk'

search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH,
    number_images=100
)

Found: 100 search results. Extracting links from 0:100
Found: 56 image links, looking for more ...
Found: 112 search results. Extracting links from 100:112
Found: 58 image links, looking for more ...
Found: 212 search results. Extracting links from 112:212
Found: 76 image links, looking for more ...
Found: 312 search results. Extracting links from 212:312
Found: 88 image links, looking for more ...
Found: 412 search results. Extracting links from 312:412
Found: 97 image links, looking for more ...
Found: 412 search results. Extracting links from 412:412
Found: 97 image links, looking for more ...
Found: 463 search results. Extracting links from 412:463
Found: 100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS5wAgaTY5lmCKtiGAP58I2W-aXeda7-SzJww&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\6c6016cb4d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT6Dss4sgG_v6FTYjTRCL5vBy8gNUw8mYoRog&

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSjnirHj91WVWbq2h5emTt8rZae3olGy4_XrA&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\a09c98ace8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTEJVaYjQa3n-Vm0ojzepntHnOvKl4_34U0LA&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\1433724a54.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQPP7ZOvOtRXBnvL9qGZp4EBArzuujIxfpd9g&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\367c256e3a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTTaxhX1n1FTCZKFjpoNY8rJwTeRaji5A3x1Q&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\7de0703619.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ8HtZU7HXOub789ZmwjF1amBnsRfIuKE1bcg&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\32a9048a7d.jpg


SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRSNIg3UQFlz4P46-GnSxJ9x_CFR41aK-AOQA&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\7c1b84a4f5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR92j0rgGTOCO8labtFoYQt1hi9z30uEqQ40w&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\3508d199b5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSIyo7IsB9-OuF72slr0oTpCZtzo6fNN9TnwA&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\f9e8ccd6df.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSVbkt-HnzYHNhIRP8NG_L8g5YurpwPgYAlIw&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\28735eb9a8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ601STz6zE87rtQ5la1aJQapXSJu4hCB3uKg&usqp=CAU - as C:/Users/Voence/Desktop/python_work/Scraping/images\elon_musk\b6fd8a273b.jpg
